In [1]:
default_final_bond_dim = 100
default_sweep_schedule_bond_dims = [default_final_bond_dim] * 4 + [
    default_final_bond_dim
] * 4
default_sweep_schedule_noise = [1e-4] * 4 + [1e-5] * 4 + [0]
default_sweep_schedule_davidson_threshold = [1e-10] * 8

In [2]:
import ferm_utils as feru
import csa_utils as csau
import openfermion as of
import numpy as np
from sdstate import *
import random
import h5py
import sys
from matrix_utils import construct_orthogonal
import pickle

ps_path = "planted_solutions/"
# File name in ps_path folder
file_name = "2_co2_6-311++G___12_9d464efb-b312-45f8-b0ba-8c42663059dc.pkl"

def construct_Hamiltonian_with_solution(path, file_name, key = ""):
    """
    Load the planted_soluiton Hamiltonian from the planted_solution, with key to be the block structure of
    encoding the Hamiltonian.
    """
    with open(ps_path + file_name, 'rb') as handle:
        dic = pickle.load(handle)
    if not key:
        key = list(dic.keys())[0]
    print(dic)
    E_min = dic[key]["E_min"]
    cas_x = dic[key]["cas_x"]
    killer = dic[key]["killer"]
    k = dic[key]["k"]
    upnum = dic[key]["upnum"]
    spin_orbs = dic[key]["spin_orbs"]
#     Number of electrons in each CAS block
    e_nums = dic[key]["e_nums"]
#     Solution in a list of sdstates, each sdstate represent the ground state with in each CAS block.
# This can be generalized into the general solution with concatenate() in sdstates, but not recommended as it 
# requires exponential time and space.
    sol = dic[key]["sol"]
#     CAS 2e tensor
    tbt = csau.get_cas_matrix(cas_x, spin_orbs, k)
#     Construct hidden 2e tensor and Hamiltonian
    killer_tbt = feru.get_chemist_tbt(killer, spin_orbs, spin_orb = True)
    killer_one_body = of.normal_ordered(killer - feru.get_ferm_op(killer_tbt, spin_orb=True))
    killer_onebody_matrix = feru.get_obt(killer_one_body, n = spin_orbs, spin_orb = True)
    killer_onebody_tbt = feru.onebody_to_twobody(killer_onebody_matrix)
    Htbt_killer = np.add(killer_tbt, killer_onebody_tbt) 
    Htbt_with_killer = np.add(tbt, Htbt_killer)
#     Set up random unitary to hide 2e tensor, the parameters here are initally set as random unitary rotations
    random_uparams = np.random.rand(upnum)
    U = construct_orthogonal(spin_orbs, random_uparams)
#     Hide 2e etensor with random unitary transformation
    Htbt_hidden = csau.cartan_orbtransf(Htbt_with_killer, U, complex = False)
#     Hidden 2e tensor without killer
    tbt_hidden = csau.cartan_orbtransf(tbt, U, complex = False)
    return tbt, tbt_hidden, Htbt_with_killer, Htbt_hidden, sol, e_nums, E_min, spin_orbs

tbt, tbt_hidden, Htbt_with_killer, Htbt_hidden, sol, e_nums, E_min, spin_orbs = construct_Hamiltonian_with_solution(ps_path, file_name)

{'8-4': {'E_min': -119.51845742309831, 'e_nums': [5, 3], 'sol': <sdstate.sdstate object at 0x1057f0910>, 'killer': 61.92201493870118 [] +
-19.630372675809255 [0^ 0] +
1.9630372675809253 [0^ 0 0^ 0] +
1.9630372675809253 [0^ 0 1^ 1] +
1.9630372675809253 [0^ 0 2^ 2] +
1.9630372675809253 [0^ 0 3^ 3] +
1.9630372675809253 [0^ 0 4^ 4] +
1.9630372675809253 [0^ 0 5^ 5] +
1.9630372675809253 [0^ 0 6^ 6] +
1.9630372675809253 [0^ 0 7^ 7] +
-0.06497416293935052 [0^ 2] +
0.02165805431311684 [0^ 2 8^ 8] +
0.02165805431311684 [0^ 2 9^ 9] +
0.02165805431311684 [0^ 2 10^ 10] +
0.02165805431311684 [0^ 2 11^ 11] +
-0.11358583646382721 [0^ 4] +
0.0378619454879424 [0^ 4 8^ 8] +
0.0378619454879424 [0^ 4 9^ 9] +
0.0378619454879424 [0^ 4 10^ 10] +
0.0378619454879424 [0^ 4 11^ 11] +
-19.630372675809255 [1^ 1] +
1.9630372675809253 [1^ 1 0^ 0] +
1.9630372675809253 [1^ 1 1^ 1] +
1.9630372675809253 [1^ 1 2^ 2] +
1.9630372675809253 [1^ 1 3^ 3] +
1.9630372675809253 [1^ 1 4^ 4] +
1.9630372675809253 [1^ 1 5^ 5] +
1.9630

NameError: name 'dic' is not defined

In [3]:
from CAS.dmrghandler.src.dmrghandler.qchem_dmrg_calc import single_qchem_dmrg_calc

In [4]:
def cas_to_dmrg(path, file_name, key=""):
    (tbt, tbt_hidden, Htbt_with_killer, Htbt_hidden, sol, e_nums,
     E_min, spin_orbs) \
        = construct_Hamiltonian_with_solution(path, file_name, key=key)

    num_unpaired_electrons = sum(e_nums)
    multiplicity = 1
    orb_sym = None
    symmetry_type = "SZ"

    init_state_bond_dimension = 30
    max_num_sweeps = 10
    energy_convergence_threshold = 1e-8
    sweep_schedule_bond_dims = default_sweep_schedule_bond_dims
    sweep_schedule_noise = default_sweep_schedule_noise
    sweep_schedule_davidson_threshold = (
        default_sweep_schedule_davidson_threshold
    )
    nuc_rep_energy = 0

    dmrg_param = {
        "factor_half_convention": True,
        "symmetry_type": symmetry_type,
        "num_threads": 1,
        "n_mkl_threads": 1,
        "num_orbitals": spin_orbs // 2,
        "num_spin_orbitals": spin_orbs,
        "num_electrons": sum(e_nums),
        "two_S": num_unpaired_electrons,
        "two_Sz": int((multiplicity - 1) / 2),
        "orb_sym": orb_sym,
        "temp_dir": "./tests/temp",
        "stack_mem": 1073741824,
        "restart_dir": "./tests/restart",
        "core_energy": nuc_rep_energy,
        "reordering_method": "none",
        "init_state_seed": 64241,  # 0 means random seed
        "initial_mps_method": "random",
        "init_state_bond_dimension": init_state_bond_dimension,
        "occupancy_hint": None,
        "full_fci_space_bool": False,
        "init_state_direct_two_site_construction_bool": False,
        "max_num_sweeps": max_num_sweeps,
        "energy_convergence_threshold": energy_convergence_threshold,
        "sweep_schedule_bond_dims": sweep_schedule_bond_dims,
        "sweep_schedule_noise": sweep_schedule_noise,
        "sweep_schedule_davidson_threshold": sweep_schedule_davidson_threshold,
        "davidson_type": None,  # Default is None, for "Normal"
        "eigenvalue_cutoff": 1e-20,
        "davidson_max_iterations": 4000,  # Default is 4000
        "davidson_max_krylov_subspace_size": 50,  # Default is 50
        "lowmem_noise_bool": False,
        "sweep_start": 0,  # Default is 0, where to start sweep
        "initial_sweep_direction": None,
        "stack_mem_ratio": 0.8,
    }
    one_body_tensor = np.zeros((spin_orbs, spin_orbs))
    result = single_qchem_dmrg_calc(one_body_tensor, tbt,
                                    dmrg_param)
    return result["dmrg_ground_state_energy"], E_min

In [ ]:
ps_path = "planted_solutions/"
# File name in ps_path folder
file_name = "2_co2_6-311++G___12_9d464efb-b312-45f8-b0ba-8c42663059dc.pkl"
tbt, tbt_hidden, Htbt_with_killer, Htbt_hidden, sol, e_nums, E_min, spin_orbs = construct_Hamiltonian_with_solution(ps_path, file_name)
print(sum(e_nums))
print(E_min)
print(spin_orbs)
print(Htbt_hidden.shape)
print(tbt_hidden.shape)
cas_to_dmrg(ps_path, file_name, key="")

{'8-4': {'E_min': -119.51845742309831, 'e_nums': [5, 3], 'sol': <sdstate.sdstate object at 0x1057cfa00>, 'killer': 61.92201493870118 [] +
-19.630372675809255 [0^ 0] +
1.9630372675809253 [0^ 0 0^ 0] +
1.9630372675809253 [0^ 0 1^ 1] +
1.9630372675809253 [0^ 0 2^ 2] +
1.9630372675809253 [0^ 0 3^ 3] +
1.9630372675809253 [0^ 0 4^ 4] +
1.9630372675809253 [0^ 0 5^ 5] +
1.9630372675809253 [0^ 0 6^ 6] +
1.9630372675809253 [0^ 0 7^ 7] +
-0.06497416293935052 [0^ 2] +
0.02165805431311684 [0^ 2 8^ 8] +
0.02165805431311684 [0^ 2 9^ 9] +
0.02165805431311684 [0^ 2 10^ 10] +
0.02165805431311684 [0^ 2 11^ 11] +
-0.11358583646382721 [0^ 4] +
0.0378619454879424 [0^ 4 8^ 8] +
0.0378619454879424 [0^ 4 9^ 9] +
0.0378619454879424 [0^ 4 10^ 10] +
0.0378619454879424 [0^ 4 11^ 11] +
-19.630372675809255 [1^ 1] +
1.9630372675809253 [1^ 1 0^ 0] +
1.9630372675809253 [1^ 1 1^ 1] +
1.9630372675809253 [1^ 1 2^ 2] +
1.9630372675809253 [1^ 1 3^ 3] +
1.9630372675809253 [1^ 1 4^ 4] +
1.9630372675809253 [1^ 1 5^ 5] +
1.9630